In [8]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torch.utils.data import TensorDataset, DataLoader
import re
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from razdel import tokenize
from navec import Navec

In [9]:
device = torch.device ( "cuda:0" if torch.cuda.is_available() else "cpu" )
print ( device )

cuda:0


In [10]:
df = pd.read_csv("../datasets/Petitions.csv")
corpus = df.sample(10000, replace=None, random_state=0)
corpus

,id,public_petition_text,reason_category
10014,3041543,На протяжении нескольких дней из крана идет гр...,Водоснабжение
29860,3296230,граффити на стенде детской площадки,Благоустройство
47582,3322077,По адресу пер. Джамбула д. 12 на фасаде МКД на...,Нарушение правил пользования общим имуществом
27828,3080599,Бумажные объявления на ТСОДД,Благоустройство
10170,3301487,ямы на дороге,Благоустройство
...,...,...,...
36775,3041604,Незаконная торговля не пресечена,Незаконная реализация товаров с торгового обор...
39271,2994297,Надписи на будке.,Благоустройство
44990,3236936,"Снег, лед. Необходимо убрать",Благоустройство
59226,3245246,Влажное подметание лестничных площадок и марше...,Содержание МКД


In [11]:
corpus_text = corpus["public_petition_text"]
corpus_text_category = corpus["reason_category"]

In [103]:
print(corpus_text)
print(type(corpus_text))

10014    На протяжении нескольких дней из крана идет гр...
29860                  граффити на стенде детской площадки
47582    По адресу пер. Джамбула д. 12 на фасаде МКД на...
27828                         Бумажные объявления на ТСОДД
10170                                        ямы на дороге
                               ...                        
36775                     Незаконная торговля не пресечена
39271                                    Надписи на будке.
44990                         Снег, лед. Необходимо убрать
59226    Влажное подметание лестничных площадок и марше...
25841    Затоплен подвал грязевой жижей, воняет. Парадн...
Name: public_petition_text, Length: 10000, dtype: object
<class 'pandas.core.series.Series'>


In [35]:
from sklearn.preprocessing import LabelEncoder

In [36]:
le = LabelEncoder()
corpus_text_category_encoded = le.fit_transform(corpus_text_category)
dict(zip(range(0, len(set(corpus_text_category_encoded))),le.inverse_transform(list(set(corpus_text_category_encoded)))))

{0: 'Благоустройство',
 1: 'Водоотведение',
 2: 'Водоснабжение',
 3: 'Кровля',
 4: 'Нарушение порядка пользования общим имуществом',
 5: 'Нарушение правил пользования общим имуществом',
 6: 'Незаконная информационная и (или) рекламная конструкция',
 7: 'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)',
 8: 'Повреждения или неисправность элементов уличной инфраструктуры',
 9: 'Подвалы',
 10: 'Санитарное состояние',
 11: 'Содержание МКД',
 12: 'Состояние рекламных или информационных конструкций',
 13: 'Фасад',
 14: 'Центральное отопление'}

In [104]:
corpus_encoded = []
for i in range(len(corpus_text)):
    corpus_encoded.append((corpus_text.iloc[i], corpus_text_category_encoded[i]))

In [16]:
navec = Navec.load('../models/navec_hudlit_v1_12B_500K_300d_100q.tar')

In [110]:
from nltk.corpus import stopwords
def prepare_data(corpus):
    morph = MorphAnalyzer()
    from nltk.corpus import stopwords
    stopwords = stopwords.words('russian')
    prepared_data = []
    for petition, label in corpus:   
        data = petition.lower()
        data1 = re.sub('[\\r|\\n]+', ' ', data)
        data2 = re.sub('[a-zA-Z]+', '', data1)
        data3 = re.sub('[0-9]+', '', data2)
        data4 = re.sub('[^\s^\w]+', '', data3)
        data5 = list(tokenize(data4))
        data6 = map(lambda x: x.text , data5)
        data7 = map(lambda x: morph.normal_forms(x)[0], data6)
        data8 = [w for w in data7 if w not in stopwords]
        data9 = [navec[x].tolist() for x in data8 if x in navec]
        if(len(data9)):
            prepared_data.append((torch.tensor(data9).type(torch.float32), label))
    return prepared_data

In [112]:
corpus_text_prep = prepare_data(corpus_encoded)

In [216]:
from torch.nn.utils.rnn import pad_sequence
corpus_text_padded = pad_sequence([tupl[0] for tupl in corpus_text_prep], True)

In [217]:
corpus_text_padded.shape

torch.Size([9964, 171, 300])

In [218]:
X_train, X_test, y_train, y_test = train_test_split(corpus_text_padded, [tupl[1] for tupl in corpus_text_prep], test_size=0.01)


In [239]:
ds = TensorDataset(X_train, torch.tensor(y_train).type(torch.int64))
dl = DataLoader(ds, batch_size=32, shuffle=True)

In [267]:
class Rnn(nn.Module):
    def __init__(self):
        super(Rnn, self).__init__()
        self.rnn = nn.RNN(300, 128)
        self.first_linear = nn.Linear(128, 64)
        self.first_activ = nn.Tanh()
        self.second_linear = nn.Linear(64, 15)
    def forward(self, x):
        y = self.rnn(x)[0]
        #y = self.first_linear(torch.index_select(y, 1, torch.tensor(y.shape[1]-1).to(device) ))
        y = self.first_linear(y.mean(dim=1))
        y = self.first_activ(y)
        y = self.second_linear(y)
        return y

In [268]:
loss = nn.CrossEntropyLoss()
model = Rnn()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)

In [269]:
epochs = 10
for epoch in range(epochs):
    batch = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer.zero_grad()
        batch+=1
        outputs = model(x_b)
        #outputs= outputs.view(x_b.shape[0],15)
        loss_value = loss(outputs, y_b)
        loss_value.backward()
        
        optimizer.step()
        
        sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 0.579595685005188


Эпоха 2, Значение функции потерь: 1.7482246160507202


Эпоха 3, Значение функции потерь: 0.9861229658126831


Эпоха 4, Значение функции потерь: 1.0973880290985107


Эпоха 5, Значение функции потерь: 0.780437171459198


Эпоха 6, Значение функции потерь: 1.572412371635437


Эпоха 7, Значение функции потерь: 0.9320122003555298


Эпоха 8, Значение функции потерь: 0.8534776568412781


Эпоха 9, Значение функции потерь: 0.8291677236557007


Эпоха 10, Значение функции потерь: 1.2620368003845215


In [270]:
answ = model.forward(X_test.to(device))

In [271]:
answ.shape

torch.Size([100, 15])

In [272]:
answ_encoded = []
for i in answ:
    answ_encoded.append(torch.argmax(i).to("cpu").item())

In [273]:
from sklearn.metrics import classification_report
print(classification_report(y_test, answ_encoded))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92        48
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         2
          11       0.64      0.97      0.77        33
          13       0.00      0.00      0.00         2

    accuracy                           0.77       100
   macro avg       0.15      0.19      0.17       100
weighted avg       0.64      0.77      0.70       100



c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [33]:
model.to("cpu")
cbow_mt = []
for i in vocab:
    cbow_mt.append(model.first_linear(model.embedder(torch.tensor(metadata[i]))).detach().numpy())

In [36]:
cbow_mt1 = pd.DataFrame(cbow_mt)

In [37]:
with open('cbow_mt.tsv','w', encoding="utf-8") as write_tsv:
    write_tsv.write(cbow_mt1.to_csv(sep='\t', index=False))